# RT2 Assignment

- Create a Jupyter Notebook to replace User Interface (Node A)
- Use widgets to let the user know the position of the robot and all targets that have been set and cancelled in the environment & the distance of the closest obstacle

## Notebook

Import the Required Libraries:

In [1]:
import jupyros as jr
import rospy
import sys
import select
import time
import math
import actionlib
import actionlib.msg
import assignment_2_2022
import assignment_2_2022.msg
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt

from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
from assignment_2_2022.msg import RobotMsg
from std_srvs.srv import *
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation

%matplotlib widget

In [2]:
goals_cancelled = 0
goals_reached = 0

Implement the widgets:

In [3]:
# Buttons for Selecting Modes

send_goal_button = widgets.Button(value = False, description = "Target Position",
                                  disabled = False, button_style = '')

cancel_goal_button = widgets.Button(value = False, description = "Cancel Position", 
                                    disabled = False, button_style = '')

In [4]:
# Numeric Widgets

x = widgets.BoundedFloatText(value = 0.0, description = 'x', min = -9.0, max = 9.0,
                            style = {'description_width': 'initial'}, step = 0.1, 
                            layout = widgets.Layout(width = '100px'))
y = widgets.BoundedFloatText(value = 0.0, description = 'y', min = -9.0, max = 9.0,
                            style = {'description_width': 'initial'}, step = 0.1, 
                            layout = widgets.Layout(width = '100px'))

In [9]:
def callback_send(msg):
    global cancelled, goal
    goal = assignment_2_2022.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x.value
    goal.target_pose.pose.position.y = y.value
    cancelled = False
    client.send_goal(goal)

In [10]:
def callback_cancel(msg):
    global cancelled, goal 
    goal = None
    cancelled = True
    client.cancel_goal()

In [11]:
send_goal_button.on_click(callback_send)
cancel_goal_button.on_click(callback_cancel)

Position Visualization:

In [12]:
class Odom_Visualizer:
    def __init__(self):
        self.fig_odom, self.ax = plt.subplots()
        # Robot Position Plot
        self.ln, = plt.plot([], [], 'ro', label = 'Robot Position')
        # Target's Position Plot
        self.goal_ln, = plt.plot([], [], 'b*', markersize = 10, label = 'Robot Goal Position')
        # Robot's Position Data Arrays
        self.x_data, self.y_data = [], []
        
    def plot_init(self):
        # Set Plot Title
        self.ax.set_title("Robot Odometry", fontsize = 20, fontweight = 'bold')
        # Set Plot Axis Labels
        self.ax.set_xlabel("X [m]", fontsize = 10, fontweight = "bold")
        self.ax.set_ylabel("Y [m]", fontsize = 10, fontweight = "bold")
        # Set Plot Axis Limits
        self.ax.set_xlim(-20, 20)
        self.ax.set_ylim(-20, 20)
        # Set Grid to True
        self.ax.grid(True)
        return self.ln, self.goal_ln
    
    def odom_callback(self, msg):
        # Callback Function used to update data
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)         
        
    def update_plot(self, frame):
        # Update Robot's Position on plot
        self.ln.set_data(self.x_data, self.y_data)  
        
        if cancelled: 
            self.goal_ln.set_data([], [])
        elif goal is not None:
            self.goal_ln.set_data(goal.target_pose.pose.position.x, goal.target_pose.pose.position.y)
        else:
            self.goal_ln.set_data([], [])
        
        return self.ln, self.goal_ln

Number of Reached/Cancelled Goals Visualization:

In [13]:
class Goal_Visualizer:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        # Setting up the values
        self.reached = 0
        self.cancelled = 0
        # Set Plot Title
        self.ax.set_title("Reached/Cancelled Goals", fontsize = 20, fontweight = "bold")
        
        self.ax.grid(axis = 'y', color = 'grey', linestyle = '-', alpha = 0.5)
        # Set Plot Labels on x-axis
        self.labels = ('Reached', 'Cancelled')
        self.x_pos = np.arange(len(self.labels))
        self.ax.set_xticks(self.x_pos)
        self.ax.set_xticklabels(self.labels)
        # Set Plot Axis Limits
        self.ax.set_ylim([0, 10])
        self.ax.set_yticks(np.arange(0, 11, 1))
        self.bar_colors = ['green', 'red']
        self.bar_plot = self.ax.bar(self.x_pos, [self.reached, self.cancelled], align = 'center', color = self.bar_colors, width = 0.2)

    def goal_callback(self, msg):
        # Get the number of reached/cancelled goals
        global goals_cancelled, goals_reached
        self.reached = (msg.status.status == 3)
        self.cancelled = (msg.status.status == 2) 

    def update_plot(self, frame):
        self.green_val = np.random.randint(0,100)
        self.red_val = np.random.randint(0,100)
        
        for i, bar in enumerate(self.bar_plot):
            if i == 0:
                bar.set_height(self.reached)
            else:
                bar.set_height(self.cancelled)
            bar.set_color(self.bar_colors[i])
        return self.bar_plot

Overall Plot of the Laser Scanner:

In [14]:
class Laser_Visualizer:
    def __init__(self):
        self.fig = plt.figure(figsize = (6,6))
        self.ax = plt.subplot(111, polar = True)
        self.ax.set_thetalim(-np.pi/2, np.pi/2)
        self.ax.set_theta_zero_location("N")
        self.laser = []
        self.angles = []
        self.ln, = self.ax.plot([], [], 'bo')
        
    def plot_init(self):
        # Set Plot Title
        self.ax.set_title("Robot Laser Scan", fontsize = 20, fontweight = 'bold')
        # Set Plot Y Label
        self.ax.set_ylabel("Wall Distance", fontsize = 10, fontweight = "bold")
        # Set Grid to True
        self.ax.grid(True)
        return self.ln
    
    def laser_callback(self, msg):
        # Callback Function used to update data
        self.angles = list(np.arange(msg.angle_min, msg.angle_max + msg.angle_increment, msg.angle_increment))
        self.laser = list(msg.ranges)
        
    def update_plot(self, frame):
        self.ax.set_rmax(20)
        self.ln.set_data(self.angles, self.laser)  
        return self.ln

Initialize the Node:

In [15]:
rospy.init_node('jupyter_notebook')
pub = rospy.Publisher("/pos_vel", RobotMsg, queue_size = 1)
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
client.wait_for_server()

True

## Main:

Set target position:

In [16]:
display(widgets.HBox([x, y]))
display(widgets.HBox([send_goal_button, cancel_goal_button]))

Plot number of cancelled/reached goals:

In [17]:
visualize_goals = Goal_Visualizer()
results = rospy.Subscriber('/reaching_goal/result', assignment_2_2022.msg.PlanningActionResult, visualize_goals.goal_callback)

animation_goal = FuncAnimation(visualize_goals.fig, visualize_goals.update_plot, interval = 1000)
plt.show(block = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plot robot position:

In [18]:
position_visualizer = Odom_Visualizer()
sub = rospy.Subscriber('/odom', Odometry, position_visualizer.odom_callback)

animation_pos = FuncAnimation(position_visualizer.fig_odom, position_visualizer.update_plot, init_func = position_visualizer.plot_init)
plt.show(block = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Laser plot:

In [19]:
laser_visualizer = Laser_Visualizer()
sub = rospy.Subscriber('/scan', LaserScan, laser_visualizer.laser_callback)

animation_laser = FuncAnimation(laser_visualizer.fig, laser_visualizer.update_plot, init_func = laser_visualizer.plot_init)
plt.show(block = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …